<a href="https://colab.research.google.com/github/sayakpaul/CI-CD-for-Model-Training/blob/main/cloud_function_trigger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## References

* https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai/blob/main/05-continuous-training.ipynb

## Setting up

In [ ]:
!pip install --upgrade -q google-cloud-pubsub

### ***Restart runtime.***

In [ ]:
!gcloud init

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
GOOGLE_CLOUD_PROJECT = "fast-ai-exploration"
GOOGLE_CLOUD_REGION = "us-central1"
GCS_BUCKET_NAME = "vertex-tfx-mlops"

PIPELINE_NAME = "penguin-vertex-training"
PIPELINE_ROOT = "gs://{}/pipeline_root/{}".format(GCS_BUCKET_NAME, PIPELINE_NAME)
PIPELINE_LOCATION = f"{PIPELINE_ROOT}/{PIPELINE_NAME}.json"
PUBSUB_TOPIC = f"trigger-{PIPELINE_NAME}"

DATA_ROOT = "gs://{}/data/{}".format(GCS_BUCKET_NAME, PIPELINE_NAME)
MODULE_ROOT = "gs://{}/pipeline_module/{}".format(GCS_BUCKET_NAME, PIPELINE_NAME)

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging

    logging.error("Please set all required parameters.")

## Create Pub/Sub Topic

In [ ]:
!gcloud pubsub topics create {PUBSUB_TOPIC}

## Deploy Cloud Function

In [ ]:
ENV_VARS=f"""\
PROJECT={GOOGLE_CLOUD_PROJECT},\
REGION={GOOGLE_CLOUD_REGION},\
GCS_PIPELINE_FILE_LOCATION={PIPELINE_LOCATION}
"""

!echo {ENV_VARS}

In [ ]:
!git clone -b dev https://github.com/sayakpaul/CI-CD-for-Model-Training --quiet
%cd CI-CD-for-Model-Training

In [ ]:
BUCKET = f'gs://{GCS_BUCKET_NAME}'
CLOUD_FUNCTION_NAME = f'trigger-{PIPELINE_NAME}-fn'

!gcloud functions deploy {CLOUD_FUNCTION_NAME} \
    --region={GOOGLE_CLOUD_REGION} \
    --trigger-topic={PUBSUB_TOPIC} \
    --runtime=python37 \
    --source=cloud_function\
    --entry-point=trigger_pipeline\
    --stage-bucket={BUCKET}\
    --update-env-vars={ENV_VARS}

# `trigger_pipeline` is the name of the function inside
# `cloud_function/main.py`

In [ ]:
import IPython

cloud_fn_url = f"https://console.cloud.google.com/functions/details/{GOOGLE_CLOUD_REGION}/{CLOUD_FUNCTION_NAME}"
html = (
    f'See the Cloud Function details <a href="{cloud_fn_url}" target="_blank">here</a>.'
)
IPython.display.display(IPython.display.HTML(html))

## Public Topic Message 

In [ ]:
from google.cloud import pubsub
import json

publish_client = pubsub.PublisherClient()
topic = f"projects/{GOOGLE_CLOUD_PROJECT}/topics/{PUBSUB_TOPIC}"
data = {"num_epochs": 3, "learning_rate": 1e-2}
message = json.dumps(data)

_ = publish_client.publish(topic, message.encode())

The pipeline should be available on the [Vertex AI Pipelines dashboard](https://console.cloud.google.com/vertex-ai/pipelines).

![](https://i.ibb.co/m9bYf3R/image.png)